In [ ]:
import os

def batch_rename(folder, prefix="img_", start_num=1):
    files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    files.sort()  

    num = start_num

    for filename in files:
        ext = os.path.splitext(filename)[1]  
        new_name = f"{prefix}{num}{ext}"
        src = os.path.join(folder, filename)
        dst = os.path.join(folder, new_name)

        os.rename(src, dst)
        print(f"{filename} -> {new_name}")

        num += 1
folder_path = "/home/pearson/Projects/ECE253/dataset"
batch_rename(folder_path, prefix="img_", start_num=1)

IMG_3834.PNG -> img_1.PNG
IMG_3835.PNG -> img_2.PNG
IMG_3836.PNG -> img_3.PNG
IMG_3837.PNG -> img_4.PNG
IMG_3838.PNG -> img_5.PNG
IMG_3839.PNG -> img_6.PNG
IMG_3840.PNG -> img_7.PNG
IMG_3841.PNG -> img_8.PNG
IMG_3842.PNG -> img_9.PNG
IMG_3843.PNG -> img_10.PNG
IMG_3844.PNG -> img_11.PNG
IMG_3845.PNG -> img_12.PNG
IMG_3846.PNG -> img_13.PNG
IMG_3847.PNG -> img_14.PNG
IMG_3848.PNG -> img_15.PNG
IMG_3849.PNG -> img_16.PNG
IMG_3850.PNG -> img_17.PNG
IMG_3851.PNG -> img_18.PNG
IMG_3852.PNG -> img_19.PNG
IMG_3853.PNG -> img_20.PNG
IMG_3854.PNG -> img_21.PNG
IMG_3855.PNG -> img_22.PNG
IMG_3856.PNG -> img_23.PNG
IMG_3857.PNG -> img_24.PNG
IMG_3858.PNG -> img_25.PNG
IMG_3859.PNG -> img_26.PNG
IMG_3860.PNG -> img_27.PNG
IMG_3861.PNG -> img_28.PNG
IMG_3862.PNG -> img_29.PNG
IMG_3865.PNG -> img_30.PNG
IMG_3866.PNG -> img_31.PNG
IMG_3867.PNG -> img_32.PNG
IMG_3868.PNG -> img_33.PNG
IMG_3869.PNG -> img_34.PNG
IMG_3870.PNG -> img_35.PNG
IMG_3872.PNG -> img_36.PNG
IMG_3873.PNG -> img_37.PNG
IMG_3874.P

In [53]:
import os
import cv2
import torch
import glob
import numpy as np  
from tqdm import tqdm
from groundingdino.util.inference import load_model, load_image, predict, annotate

def process_single_folder(model, input_dir, output_dir, config):
    """
    处理单个文件夹，并计算详细统计数据。
    """
    # 解包配置
    text_prompt = config["text_prompt"]
    box_threshold = config["box_threshold"]
    text_threshold = config["text_threshold"]
    device = config["device"]

    os.makedirs(output_dir, exist_ok=True)
    
    # 获取图片列表
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG']
    image_paths = []
    for ext in image_extensions:
        image_paths.extend(glob.glob(os.path.join(input_dir, ext)))
    
    print(f"\n{'-'*60}")
    print(f"Processing Folder: {input_dir}")
    print(f"Output Directory:  {output_dir}")
    print(f"Images Found:      {len(image_paths)}")
    print(f"{'-'*60}")

    if not image_paths:
        print("No images found. Skipping.")
        return

    # --- 数据容器 ---
    # 1. 存储该文件夹下所有图片的所有 Score，用于计算文件夹级别的 Mean 和 Variance
    all_scores_in_folder = []
    # 2. 存储每张图片的详细信息 (如果你后续需要导出到Excel，可以用这个列表)
    per_image_stats = []
    # 3. 文件夹总框数计数器
    folder_total_boxes = 0

    # --- 循环处理图片 ---
    for img_path in tqdm(image_paths, desc="Progress"):
        filename = os.path.basename(img_path)
        
        try:
            image_source, image = load_image(img_path)

            boxes, logits, phrases = predict(
                model=model,
                image=image,
                caption=text_prompt,
                box_threshold=box_threshold,
                text_threshold=text_threshold,
                device=device
            )

            # 保存可视化结果
            annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
            cv2.imwrite(os.path.join(output_dir, f"annotated_{filename}"), annotated_frame)

            # --- 收集单张图片统计信息 ---
            current_count = len(boxes)
            # 将 Tensor 转为 list
            current_scores = logits.tolist() 

            # 存储单张图片数据
            per_image_stats.append({
                "filename": filename,
                "box_count": current_count,
                "scores": current_scores
            })

            # 更新文件夹级别的汇总数据
            folder_total_boxes += current_count
            all_scores_in_folder.extend(current_scores)

        except Exception as e:
            print(f"Error processing {filename}: {e}")

    # --- 计算文件夹级别的统计 ---
    print(f"\n>>> STATS REPORT FOR FOLDER: {os.path.basename(input_dir)}")
    
    # 1. 总框数
    print(f"Total Boxes Detected: {folder_total_boxes}")

    # 2. 平均分和方差 (需要判断列表是否为空)
    if len(all_scores_in_folder) > 0:
        # 使用 numpy 计算平均值和方差
        folder_mean = np.mean(all_scores_in_folder)
        folder_variance = np.var(all_scores_in_folder)
        
        print(f"Average Confidence:   {folder_mean:.4f}")
        print(f"Confidence Variance:  {folder_variance:.4f}")
    else:
        print("Average Confidence:   N/A (No detections)")
        print("Confidence Variance:  N/A")
    
    # (可选) 打印每张图片的详情
    # for stat in per_image_stats:
    #     print(f"  File: {stat['filename']}, Count: {stat['box_count']}, Scores: {stat['scores']}")

    print(f"{'='*60}\n")


def main():
    # --- 1. 配置区域 (Configuration) ---
    # 将所有变量定义在这里，不使用全局变量
    cfg = {
        "config_path": "groundingdino/config/GroundingDINO_SwinT_OGC.py",
        "weights_path": "/home/pearson/Projects/ECE253/GroundingDINO/weights/groundingdino_swint_ogc.pth",
        "text_prompt": "person . car .",
        "box_threshold": 0.35,
        "text_threshold": 0.25,
        "device": "cpu"
    }

    # 定义要处理的文件夹列表 [(Input, Output), (Input, Output)]
    folders_to_process = [
        (
            "../dataset", 
            "../results/annoted/original/"
        ),
        (
            "../results/CLAHE/", 
            "../results/annoted/CLAHE/"
        ),
        (
            "../results/CLAHE_Wiener/", 
            "../results/annoted/CLAHE_Wiener/"
        ),
        (
            "../results/CLAHE_NAFNet/", 
            "../results/annoted/CLAHE_NAFNet/"
        ),
        (
            "../results/ZeroDCE/", 
            "../results/annoted/ZeroDCE/"
        ),
        (
            "../results/ZeroDCE_Wiener/", 
            "../results/annoted/ZeroDCE_Wiener/"
        ),
        (
            "../results/ZeroDCE_NAFNet/", 
            "../results/annoted/ZeroDCE_NAFNet/"
        ),
        # 在这里添加第二个文件夹...
        # ("/path/to/input2", "/path/to/output2"),
    ]

    # --- 2. 加载模型 (只加载一次) ---
    print(f"Loading Model from {cfg['weights_path']} ...")
    model = load_model(cfg["config_path"], cfg["weights_path"])

    # --- 3. 执行批处理 ---
    for input_dir, output_dir in folders_to_process:
        process_single_folder(model, input_dir, output_dir, cfg)

    print("All batch processing complete.")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'groundingdino'

## CLAHE

### function:

In [19]:
import os
import glob
import cv2

def clahe_enhance_image(input_path, output_path,
                        clip_limit=4.0,
                        tile_grid_size=(8, 8)):
    img = cv2.imread(input_path, cv2.IMREAD_COLOR)
    if img is None:
        print(f"[WARN] Cannot read image: {input_path}")
        return False

    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    cl = clahe.apply(l)

    merged = cv2.merge((cl, a, b))
    output = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, output)
    print(f"[OK] {input_path} -> {output_path}")
    return True


def batch_clahe(input_root, output_root,
                exts=("*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"),
                clip_limit=4.0,
                tile_grid_size=(8, 8)):
    img_paths = []
    for ext in exts:
        img_paths.extend(glob.glob(os.path.join(input_root, "**", ext), recursive=True))

    print(f"Found {len(img_paths)} images")

    for in_path in img_paths:
        rel_path = os.path.relpath(in_path, input_root)
        out_path = os.path.join(output_root, rel_path)

        clahe_enhance_image(
            in_path,
            out_path,
            clip_limit=clip_limit,
            tile_grid_size=tile_grid_size
        )

### Run:

In [20]:
clahe_input_root = "../dataset"
clahe_output_root = "../results/CLAHE"

batch_clahe(clahe_input_root, clahe_output_root,
            clip_limit=4.0,
            tile_grid_size=(8, 8))

Found 169 images
[OK] ../dataset/img_1.PNG -> ../results/CLAHE/img_1.PNG
[OK] ../dataset/img_2.PNG -> ../results/CLAHE/img_2.PNG
[OK] ../dataset/img_3.PNG -> ../results/CLAHE/img_3.PNG
[OK] ../dataset/img_4.PNG -> ../results/CLAHE/img_4.PNG
[OK] ../dataset/img_5.PNG -> ../results/CLAHE/img_5.PNG
[OK] ../dataset/img_6.PNG -> ../results/CLAHE/img_6.PNG
[OK] ../dataset/img_7.PNG -> ../results/CLAHE/img_7.PNG
[OK] ../dataset/img_8.PNG -> ../results/CLAHE/img_8.PNG
[OK] ../dataset/img_9.PNG -> ../results/CLAHE/img_9.PNG
[OK] ../dataset/img_10.PNG -> ../results/CLAHE/img_10.PNG
[OK] ../dataset/img_11.PNG -> ../results/CLAHE/img_11.PNG
[OK] ../dataset/img_12.PNG -> ../results/CLAHE/img_12.PNG
[OK] ../dataset/img_13.PNG -> ../results/CLAHE/img_13.PNG
[OK] ../dataset/img_14.PNG -> ../results/CLAHE/img_14.PNG
[OK] ../dataset/img_15.PNG -> ../results/CLAHE/img_15.PNG
[OK] ../dataset/img_16.PNG -> ../results/CLAHE/img_16.PNG
[OK] ../dataset/img_17.PNG -> ../results/CLAHE/img_17.PNG
[OK] ../dataset

## ZeroDCE

### function:

In [21]:
import torch
import torch.nn as nn
import torchvision
import torch.backends.cudnn as cudnn
import torch.optim
import os
import sys
import argparse
import time
from ZeroDCE import dataloader, model
# import model
import numpy as np
from torchvision import transforms
from PIL import Image
import glob
import time


def zerodce_lowlight(image_path, in_root, out_root, net, scale_factor=12, device="cuda"):
    data_lowlight = Image.open(image_path).convert("RGB")
    data_lowlight = np.asarray(data_lowlight) / 255.0
    data_lowlight = torch.from_numpy(data_lowlight).float()

    h = (data_lowlight.shape[0] // scale_factor) * scale_factor
    w = (data_lowlight.shape[1] // scale_factor) * scale_factor
    data_lowlight = data_lowlight[0:h, 0:w, :]
    data_lowlight = data_lowlight.permute(2, 0, 1).unsqueeze(0).to(device)

    start = time.time()
    enhanced_image, params_maps = net(data_lowlight)
    end_time = time.time() - start
    print("Inference time: {:.4f}s".format(end_time))

    rel_path = os.path.relpath(image_path, in_root)
    result_path = os.path.join(out_root, rel_path)

    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    torchvision.utils.save_image(enhanced_image, result_path)
    print("Saved to:", result_path)

    return end_time

### Run:

In [22]:
in_dir = "../dataset"
out_dir = "../results/ZeroDCE"
zerodce_model_weights_path = './ZeroDCE/trained_model/Epoch99.pth'
    
with torch.no_grad():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)

    scale_factor = 12

    os.makedirs(out_dir, exist_ok=True)

    DCE_net = model.enhance_net_nopool(scale_factor).to(device)
    DCE_net.load_state_dict(torch.load(zerodce_model_weights_path, map_location=device))
    DCE_net.eval()

    exts = ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]
    test_list = []
    for ext in exts:
        test_list.extend(glob.glob(os.path.join(in_dir, ext)))

    sum_time = 0.0

    for image in test_list:
        print("Processing:", image)
        sum_time += zerodce_lowlight(image, in_dir, out_dir, DCE_net, scale_factor=scale_factor, device=device)

    print("Total time:", sum_time)
    print("Average per image:", sum_time / max(len(test_list), 1))

Using device: cuda
Processing: ../dataset/img_1.PNG


/tmp/ipykernel_4065148/606138989.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  DCE_net.load_state_dict(torch.load(zerodce_model_weights_path, map_location=device))


Inference time: 0.0064s
Saved to: ../results/ZeroDCE/img_1.PNG
Processing: ../dataset/img_2.PNG
Inference time: 0.0012s
Saved to: ../results/ZeroDCE/img_2.PNG
Processing: ../dataset/img_3.PNG
Inference time: 0.0012s
Saved to: ../results/ZeroDCE/img_3.PNG
Processing: ../dataset/img_4.PNG
Inference time: 0.0012s
Saved to: ../results/ZeroDCE/img_4.PNG
Processing: ../dataset/img_5.PNG
Inference time: 0.0015s
Saved to: ../results/ZeroDCE/img_5.PNG
Processing: ../dataset/img_6.PNG
Inference time: 0.0013s
Saved to: ../results/ZeroDCE/img_6.PNG
Processing: ../dataset/img_7.PNG
Inference time: 0.0012s
Saved to: ../results/ZeroDCE/img_7.PNG
Processing: ../dataset/img_8.PNG
Inference time: 0.0012s
Saved to: ../results/ZeroDCE/img_8.PNG
Processing: ../dataset/img_9.PNG
Inference time: 0.0012s
Saved to: ../results/ZeroDCE/img_9.PNG
Processing: ../dataset/img_10.PNG
Inference time: 0.0014s
Saved to: ../results/ZeroDCE/img_10.PNG
Processing: ../dataset/img_11.PNG
Inference time: 0.0023s
Saved to: ../

## NAFNet

### Process:

```bash
cd NAFNet
```

run test.ipynb

## Wiener filter

In [ ]:
import os
import re
import cv2
import numpy as np


def wiener_deblur(img, kernel, K=0.01):
    # img: 2D 单通道
    img_fft = np.fft.fft2(img)
    kernel_fft = np.fft.fft2(kernel, s=img.shape[:2])
    kernel_fft_conj = np.conj(kernel_fft)
    wiener_filter = kernel_fft_conj / (kernel_fft * kernel_fft_conj + K)
    result_fft = img_fft * wiener_filter
    result = np.abs(np.fft.ifft2(result_fft))
    return result


def motion_kernel(length=15, angle=0):
    kernel = np.zeros((length, length))
    kernel[length // 2, :] = 1
    M = cv2.getRotationMatrix2D((length / 2, length / 2), angle, 1)
    kernel = cv2.warpAffine(kernel, M, (length, length))
    kernel /= kernel.sum()
    return kernel


def natural_key(s):

    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(r'(\d+)', s)]


def process_image_file(in_path, out_path, kernel, K=0.01):
    img = cv2.imread(in_path, cv2.IMREAD_COLOR)
    if img is None:
        print(f'[WARN] Failed to read image: {in_path}')
        return


    b, g, r = cv2.split(img)
    b_res = wiener_deblur(b, kernel, K)
    g_res = wiener_deblur(g, kernel, K)
    r_res = wiener_deblur(r, kernel, K)


    result = cv2.merge([
        np.clip(b_res, 0, 255).astype(np.uint8),
        np.clip(g_res, 0, 255).astype(np.uint8),
        np.clip(r_res, 0, 255).astype(np.uint8),
    ])

    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    cv2.imwrite(out_path, result)


def batch_wiener_deblur(
    input_dir,
    output_dir,
    length=3,
    angle=0,
    K=0.01,
    exts=('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')
):
    kernel = motion_kernel(length=length, angle=angle)

    all_imgs = []
    for root, _, files in os.walk(input_dir):
        for name in files:
            ext = os.path.splitext(name)[1].lower()
            if ext in exts:
                in_path = os.path.join(root, name)
                rel_path = os.path.relpath(in_path, input_dir)
                out_path = os.path.join(output_dir, rel_path)
                all_imgs.append((in_path, out_path))

    # all_imgs = sorted(all_imgs, key=lambda x: natural_key(x[0]))

    print(f'Found {len(all_imgs)} images in {input_dir}')

    for idx, (in_path, out_path) in enumerate(all_imgs):
        print(f'[{idx+1}/{len(all_imgs)}] {in_path} -> {out_path}')
        process_image_file(in_path, out_path, kernel, K)



In [50]:
input_dir = '/home/pearson/Projects/ECE253/results/ZeroDCE/'        
output_dir = '/home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/' 

batch_wiener_deblur(
    input_dir=input_dir,
    output_dir=output_dir,
    length=5,
    angle=0,
    K=0.001
)

print('All done.')

Found 169 images in /home/pearson/Projects/ECE253/results/ZeroDCE/
[1/169] /home/pearson/Projects/ECE253/results/ZeroDCE/img_1.PNG -> /home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/img_1.PNG
[2/169] /home/pearson/Projects/ECE253/results/ZeroDCE/img_2.PNG -> /home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/img_2.PNG
[3/169] /home/pearson/Projects/ECE253/results/ZeroDCE/img_3.PNG -> /home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/img_3.PNG
[4/169] /home/pearson/Projects/ECE253/results/ZeroDCE/img_4.PNG -> /home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/img_4.PNG
[5/169] /home/pearson/Projects/ECE253/results/ZeroDCE/img_5.PNG -> /home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/img_5.PNG
[6/169] /home/pearson/Projects/ECE253/results/ZeroDCE/img_6.PNG -> /home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/img_6.PNG
[7/169] /home/pearson/Projects/ECE253/results/ZeroDCE/img_7.PNG -> /home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/img_7.PNG
[8/169] /home/pearson/P

In [51]:
input_dir = '/home/pearson/Projects/ECE253/results/CLAHE/'          
output_dir = '/home/pearson/Projects/ECE253/results/CLAHE_Wiener/'  

batch_wiener_deblur(
    input_dir=input_dir,
    output_dir=output_dir,
    length=5,
    angle=0,
    K=0.001
)

print('All done.')

Found 169 images in /home/pearson/Projects/ECE253/results/CLAHE/
[1/169] /home/pearson/Projects/ECE253/results/CLAHE/img_1.PNG -> /home/pearson/Projects/ECE253/results/CLAHE_Wiener/img_1.PNG
[2/169] /home/pearson/Projects/ECE253/results/CLAHE/img_2.PNG -> /home/pearson/Projects/ECE253/results/CLAHE_Wiener/img_2.PNG
[3/169] /home/pearson/Projects/ECE253/results/CLAHE/img_3.PNG -> /home/pearson/Projects/ECE253/results/CLAHE_Wiener/img_3.PNG
[4/169] /home/pearson/Projects/ECE253/results/CLAHE/img_4.PNG -> /home/pearson/Projects/ECE253/results/CLAHE_Wiener/img_4.PNG
[5/169] /home/pearson/Projects/ECE253/results/CLAHE/img_5.PNG -> /home/pearson/Projects/ECE253/results/CLAHE_Wiener/img_5.PNG
[6/169] /home/pearson/Projects/ECE253/results/CLAHE/img_6.PNG -> /home/pearson/Projects/ECE253/results/CLAHE_Wiener/img_6.PNG
[7/169] /home/pearson/Projects/ECE253/results/CLAHE/img_7.PNG -> /home/pearson/Projects/ECE253/results/CLAHE_Wiener/img_7.PNG
[8/169] /home/pearson/Projects/ECE253/results/CLAHE/i

## Others

In [52]:
# import cv2
# import numpy as np

# def wiener_deblur(img, kernel, K=0.01):
#     img_fft = np.fft.fft2(img)
#     kernel_fft = np.fft.fft2(kernel, s=img.shape[:2])
#     kernel_fft_conj = np.conj(kernel_fft)
#     wiener_filter = kernel_fft_conj / (kernel_fft * kernel_fft_conj + K)
#     result_fft = img_fft * wiener_filter
#     result = np.abs(np.fft.ifft2(result_fft))
#     return result

# def motion_kernel(length=15, angle=0):
#     kernel = np.zeros((length, length))
#     kernel[length // 2, :] = 1
#     M = cv2.getRotationMatrix2D((length / 2, length / 2), angle, 1)
#     kernel = cv2.warpAffine(kernel, M, (length, length))
#     kernel /= kernel.sum()
#     return kernel

# img = cv2.imread("/home/pearson/Projects/ECE253/results/ZeroDCE/img_1.PNG", cv2.IMREAD_COLOR)

# kernel = motion_kernel(3, 0)

# b, g, r = cv2.split(img)
# b_res = wiener_deblur(b, kernel)
# g_res = wiener_deblur(g, kernel)
# r_res = wiener_deblur(r, kernel)


# result = cv2.merge([
#     np.clip(b_res, 0, 255).astype(np.uint8),
#     np.clip(g_res, 0, 255).astype(np.uint8),
#     np.clip(r_res, 0, 255).astype(np.uint8),
# ])

# cv2.imwrite("/home/pearson/Projects/ECE253/results/1_color.png", result)

In [ ]:
# import cv2
# import numpy as np
# from skimage.restoration import denoise_tv_chambolle


# def fft_convolve2d(x, k):

#     H, W = x.shape
#     h, w = k.shape

#     pad_k = np.zeros_like(x)
#     pad_k[:h, :w] = k

#     X = np.fft.fft2(x)
#     K = np.fft.fft2(pad_k)
#     Y = X * K
#     y = np.fft.ifft2(Y).real
#     return y


# def estimate_image(f, k, lam=0.01, tv_weight=0.1, n_inner=3):

#     H, W = f.shape
#     kh, kw = k.shape

#     pad_k = np.zeros_like(f)
#     pad_k[:kh, :kw] = k

#     K = np.fft.fft2(pad_k)
#     K_conj = np.conj(K)
#     F = np.fft.fft2(f)

#     U = (K_conj * F) / (K_conj * K + lam + 1e-8)
#     u = np.fft.ifft2(U).real

#     for _ in range(n_inner):
#         u = denoise_tv_chambolle(u, weight=tv_weight, channel_axis=None)

#     u = np.clip(u, 0.0, 1.0)
#     return u


# def estimate_kernel(f, u, k_size=15, beta=1e-3):

#     H, W = f.shape


#     margin = k_size * 2
#     ys = margin
#     ye = H - margin
#     xs = margin
#     xe = W - margin
#     if ye <= ys or xe <= xs:

#         f_c = f
#         u_c = u
#     else:
#         f_c = f[ys:ye, xs:xe]
#         u_c = u[ys:ye, xs:xe]

#     pad_u = np.zeros_like(f)
#     pad_u[:u_c.shape[0], :u_c.shape[1]] = u_c

#     U = np.fft.fft2(pad_u)
#     F = np.fft.fft2(f)


#     K = (np.conj(U) * F) / (np.conj(U) * U + beta + 1e-8)
#     k_full = np.fft.ifft2(K).real


#     cy, cx = k_full.shape[0] // 2, k_full.shape[1] // 2
#     half = k_size // 2
#     k = k_full[cy-half:cy+half+1, cx-half:cx+half+1]
#     k = np.maximum(k, 0.0)
#     s = k.sum()
#     if s > 0:
#         k /= s
#     else:
#         k = np.ones((k_size, k_size), dtype=np.float32)
#         k /= k.sum()
#     return k


# def blind_deblur_variational_gray(
#         img_gray_uint8,
#         k_size=15,
#         lam=0.01,
#         tv_weight=0.1,
#         beta=1e-3,
#         n_iters=8):

#     f = img_gray_uint8.astype(np.float32) / 255.0


#     k = np.ones((k_size, k_size), dtype=np.float32)
#     k /= k.sum()


#     u = f.copy()

#     for it in range(n_iters):
#         print(f"[Iter {it+1}/{n_iters}]")

#         u = estimate_image(f, k, lam=lam, tv_weight=tv_weight, n_inner=3)

#         k = estimate_kernel(f, u, k_size=k_size, beta=beta)

#         print(f"  kernel sum={k.sum():.4f}, min={k.min():.4e}, max={k.max():.4e}")

#     u_final = estimate_image(f, k, lam=lam, tv_weight=tv_weight, n_inner=5)
#     u_uint8 = np.clip(u_final * 255.0, 0, 255).astype(np.uint8)
#     return u_uint8, k



# def blind_deblur_variational_color(
#         img_bgr_uint8,
#         k_size=15,
#         lam=0.01,
#         tv_weight=0.1,
#         beta=1e-3,
#         n_iters=8):

#     # BGR -> YCrCb
#     ycrcb = cv2.cvtColor(img_bgr_uint8, cv2.COLOR_BGR2YCrCb)
#     Y, Cr, Cb = cv2.split(ycrcb)

#     Y_deblur, k_est = blind_deblur_variational_gray(
#         Y,
#         k_size=k_size,
#         lam=lam,
#         tv_weight=tv_weight,
#         beta=beta,
#         n_iters=n_iters
#     )


#     ycrcb_deblur = cv2.merge([Y_deblur, Cr, Cb])
#     bgr_deblur = cv2.cvtColor(ycrcb_deblur, cv2.COLOR_YCrCb2BGR)

#     return bgr_deblur, k_est




In [ ]:

# img_path = "/home/pearson/Projects/ECE253/results/CLAHE/img_1.PNG"
# out_img_path = "/home/pearson/Projects/ECE253/results/blind_var_deblur_color.png"
# out_kernel_path = "/home/pearson/Projects/ECE253/results/blind_var_kernel.png"


# img_color = cv2.imread(img_path, cv2.IMREAD_COLOR)
# if img_color is None:
#     raise FileNotFoundError(f"Cannot read image: {img_path}")


# deblur_bgr, k_est = blind_deblur_variational_color(
#     img_color,
#     k_size=15,      
#     lam=0.01,        
#     tv_weight=0.08,  
#     beta=1e-3,       
#     n_iters=8        
# )

# cv2.imwrite(out_img_path, deblur_bgr)
# print(f"Deblurred color image saved to: {out_img_path}")

# k_vis = k_est / (k_est.max() + 1e-8)
# k_vis = (k_vis * 255).astype(np.uint8)
# cv2.imwrite(out_kernel_path, k_vis)
# print(f"Estimated kernel visualization saved to: {out_kernel_path}")

[Iter 1/8]
  kernel sum=1.0000, min=0.0000e+00, max=2.5170e-02
[Iter 2/8]
  kernel sum=1.0000, min=0.0000e+00, max=3.1710e-02
[Iter 3/8]
  kernel sum=1.0000, min=0.0000e+00, max=3.2870e-02
[Iter 4/8]
  kernel sum=1.0000, min=0.0000e+00, max=3.3374e-02
[Iter 5/8]
  kernel sum=1.0000, min=0.0000e+00, max=2.8127e-02
[Iter 6/8]
  kernel sum=1.0000, min=0.0000e+00, max=3.3828e-02
[Iter 7/8]
  kernel sum=1.0000, min=0.0000e+00, max=4.1067e-02
[Iter 8/8]
  kernel sum=1.0000, min=0.0000e+00, max=3.4109e-02
Deblurred color image saved to: /home/pearson/Projects/ECE253/results/blind_var_deblur_color.png
Estimated kernel visualization saved to: /home/pearson/Projects/ECE253/results/blind_var_kernel.png
